In [ ]:
###BSW
import pandas as pd

data = pd.read_csv('../public/cpi.csv')

# data.columns = ['dummy', '연도'] + [str(year) for year in range(1970, 2024)]

data = data.transpose() # 행 열을 돌려 반전시킨다

# data = data.drop('dummy', axis=0)

for i in range(2, len(data.columns)):
    del data[i]

data = data.drop('단위:', axis=0)
data = data.drop('2020=100, %', axis=0)
data = data.drop('2023', axis=0)

# data = data.rename_axis('연도', inplace=True)

# data = data.rename(columns={'0': '소비자물가총지수'})

data.columns = ['소비자물가지수', '소비자물가상승률']


data

# ###지피티
# import pandas as pd
# 
# # CSV 파일 읽기
# data = pd.read_csv('../public/cpi.csv')
# 
# # 행/열 반전
# data = data.transpose()
# 
# # 필요 없는 행 제거 (기본적으로 index 이름이 행 이름이 됨)
# data = data.drop(['단위:', '2020=100, %', '2023'], axis=0)
# 
# # 열 이름 재지정
# data.columns = ['소비자물가지수', '소비자물가상승률']
# 
# # 결과 출력
# print(data)

In [ ]:
print(data)